## Data exploration on new databased uplodaded to Mongo

In [3]:
import pandas as pd 
import folium
from folium.plugins import HeatMap
import pymongo
from pymongo import MongoClient
from pandas.io.json import json_normalize

In [4]:
# Now that we have our data cleaned and loaded in mongodb as geospatial data, we proceed to continue our anlysis
# with the proximity to our selected companies and criteria


In [5]:
for_map = pd.read_json('afines.json')
for_map.columns

Index(['category_code', 'country_code', 'founded_year', 'geoloc', 'latitude',
       'longitude', 'name', 'number_of_employees', 'total_money_raised',
       'total_offices'],
      dtype='object')

In [6]:
def points(col_lat, col_lon): #insert values as (df[col_lat], df[col_lon])
    statmad = for_map[['latitude', 'longitude']]
    statmadlist = statmad.values.tolist()
    popup = for_map[['name']]
    statmap = folium.Map(zoom_start=11) 
    for point in range(0, len(statmadlist)):
        folium.Marker(statmadlist[point], popup=popup.iloc[point,0]).add_to(statmap)
    return statmap    

points(for_map['latitude'], for_map['longitude'])

In [7]:
# Let's star working with our new dataset at mongo:

client = MongoClient('mongodb://localhost:27017/')
db = client.companies
mongodata = db.selected.find()

df = pd.DataFrame(mongodata)
display(df.head())

,_id,category_code,country_code,founded_year,geoloc,latitude,longitude,name,number_of_employees,total_money_raised,total_offices
0,5d277baf5370f8e6fc94d89a,search,USA,2006,"{'type': 'Point', 'coordinates': [-122.395289,...",37.778613,-122.395289,Powerset,60,$22.5M,1
1,5d277baf5370f8e6fc94d89b,search,USA,2004,"{'type': 'Point', 'coordinates': [-122.402195,...",37.786183,-122.402195,Yelp,800,$90M,1
2,5d277baf5370f8e6fc94d89c,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-118.468707,...",33.991101,-118.468707,KickApps,72,$32M,4
3,5d277baf5370f8e6fc94d89d,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-73.9843328,...",40.751661,-73.984333,KickApps,72,$32M,4
4,5d277baf5370f8e6fc94d89e,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-122.4016144...",37.789109,-122.401614,KickApps,72,$32M,4


In [8]:
# We still have some NaN on our db:
df.dropna(inplace = True)
display(df.head())

,_id,category_code,country_code,founded_year,geoloc,latitude,longitude,name,number_of_employees,total_money_raised,total_offices
0,5d277baf5370f8e6fc94d89a,search,USA,2006,"{'type': 'Point', 'coordinates': [-122.395289,...",37.778613,-122.395289,Powerset,60,$22.5M,1
1,5d277baf5370f8e6fc94d89b,search,USA,2004,"{'type': 'Point', 'coordinates': [-122.402195,...",37.786183,-122.402195,Yelp,800,$90M,1
2,5d277baf5370f8e6fc94d89c,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-118.468707,...",33.991101,-118.468707,KickApps,72,$32M,4
3,5d277baf5370f8e6fc94d89d,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-73.9843328,...",40.751661,-73.984333,KickApps,72,$32M,4
4,5d277baf5370f8e6fc94d89e,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-122.4016144...",37.789109,-122.401614,KickApps,72,$32M,4


In [10]:
# With these two functions we aim to transform our geolocation data to list in order to make a geoquery to find how many
# companies we have surrounding each og them in a given radius

def cambiar (col):
    lista = []   
    for e in range(len(df)):
        lista.append(df.geoloc[e])
    return lista

def findNear(lista,rad):
    return list(db.selected.find({
        "geoloc": {
         "$near": {
           "$geometry": lista,
           "$maxDistance": rad,
         }
       }
    }))

In [11]:
lista = cambiar(df["geoloc"]) # We apply function "cambiar"

In [26]:
# we now create a list (and then a column) of the number of companies close to each of them in 1500 meters
lst =[] 
rad = 1500 # meters

for e in range(len(df)):
    num = findNear(lista[e],rad)
    lst.append(len(num))
print(lst)

[21, 27, 1, 13, 27, 7, 1, 4, 2, 1, 2, 7, 4, 2, 10, 2, 1, 6, 1, 4, 19, 27, 1, 3, 6, 23, 1, 1, 10, 1, 1, 12, 3, 27, 27, 1, 6, 2, 1, 1, 1, 1, 2, 7, 1, 1, 24, 5, 1, 2, 1, 4, 5, 2, 4, 1, 1, 21, 11, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 6, 1, 2, 7, 4, 3, 1, 1, 12, 3, 1, 1, 5, 10, 23, 1, 1, 1, 1, 27, 1, 1, 6, 1, 12, 1, 1, 2, 1, 6, 1, 1, 1, 2, 12, 1, 2, 10, 7, 5, 1, 6, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 26, 5, 2, 1, 1, 1, 1, 3, 6, 1, 1, 1, 1, 1, 1, 2, 11, 7, 25, 23, 2, 27, 2, 8, 5, 27, 13, 2, 5, 23, 27, 5, 1, 27, 5, 23, 2, 1, 1, 1, 4, 5, 2, 2, 3, 27, 1, 1, 4, 1, 1, 1, 1, 27, 2, 1, 6, 1, 2, 1, 1, 1, 2, 2, 2, 2, 25, 1, 1, 1, 1, 27, 4, 2, 1, 1, 3, 1, 2, 1, 3, 1, 1, 2, 1, 2, 1, 1, 1, 3, 1, 1, 27, 2, 1, 2, 26, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 12, 1, 1, 3, 1, 1, 12]


In [29]:
df["nearest_offices"] = lst

df = df.sort_values(by = "nearest_offices", ascending = False)
df.reset_index(inplace=True)
display(df.head())

,index,_id,category_code,country_code,founded_year,geoloc,latitude,longitude,name,number_of_employees,total_money_raised,total_offices,nearest_offices
0,51,5d277baf5370f8e6fc94d8cd,enterprise,USA,2006,"{'type': 'Point', 'coordinates': [-122.158804,...",37.439380,-122.158804,Gigya,230,$69.8M,1,27
1,20,5d277baf5370f8e6fc94d8ae,web,USA,2006,"{'type': 'Point', 'coordinates': [-122.39071, ...",37.779507,-122.390710,Eventbrite,200,$140M,1,27
2,167,5d277baf5370f8e6fc94d941,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-121.967223,...",37.369445,-121.967223,Apigee,65,$107M,1,27
3,26,5d277baf5370f8e6fc94d8b4,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-104.988123,...",39.755092,-104.988123,NewsGator,90,$39.3M,1,27
4,190,5d277baf5370f8e6fc94d958,advertising,USA,2006,"{'type': 'Point', 'coordinates': [-105.278785,...",40.058448,-105.278785,Intela,70,$13.3M,4,27


In [18]:
# Let's save our df for 1500 meter and create few more with different radius

In [30]:
df.to_csv("for_api_1500.csv")

In [31]:
# we now create a list (and then a column) of the number of companies close to each of them in 500 meters

lst =[] 
rad = 500 # meters

for e in range(len(df)):
    num = findNear(lista[e],rad)
    lst.append(len(num))
print(lst)

df["nearest_offices"] = lst

df_500 = df.sort_values(by = "nearest_offices", ascending = False)
df_500.reset_index(inplace=True)

display(df.head())

[2, 12, 1, 2, 16, 1, 1, 2, 1, 1, 2, 3, 1, 1, 2, 2, 1, 2, 1, 1, 3, 15, 1, 2, 1, 8, 1, 1, 2, 1, 1, 2, 3, 15, 3, 1, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 11, 2, 1, 2, 1, 1, 4, 1, 3, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 3, 3, 1, 1, 1, 5, 3, 1, 1, 1, 3, 7, 1, 1, 1, 1, 13, 1, 1, 4, 1, 5, 1, 1, 1, 1, 2, 1, 1, 1, 2, 5, 1, 1, 1, 2, 1, 1, 4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 16, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 2, 4, 1, 5, 2, 16, 2, 1, 3, 16, 4, 2, 3, 5, 14, 3, 1, 16, 3, 5, 2, 1, 1, 1, 3, 1, 1, 1, 1, 14, 1, 1, 2, 1, 1, 1, 1, 16, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 13, 1, 1, 1, 1, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 2, 1, 1, 14, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 4]


,index,_id,category_code,country_code,founded_year,geoloc,latitude,longitude,name,number_of_employees,total_money_raised,total_offices,nearest_offices
0,51,5d277baf5370f8e6fc94d8cd,enterprise,USA,2006,"{'type': 'Point', 'coordinates': [-122.158804,...",37.439380,-122.158804,Gigya,230,$69.8M,1,2
1,20,5d277baf5370f8e6fc94d8ae,web,USA,2006,"{'type': 'Point', 'coordinates': [-122.39071, ...",37.779507,-122.390710,Eventbrite,200,$140M,1,12
2,167,5d277baf5370f8e6fc94d941,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-121.967223,...",37.369445,-121.967223,Apigee,65,$107M,1,1
3,26,5d277baf5370f8e6fc94d8b4,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-104.988123,...",39.755092,-104.988123,NewsGator,90,$39.3M,1,2
4,190,5d277baf5370f8e6fc94d958,advertising,USA,2006,"{'type': 'Point', 'coordinates': [-105.278785,...",40.058448,-105.278785,Intela,70,$13.3M,4,16


In [33]:
df_500.to_csv("for_api_500.csv")

In [34]:
# we now create a list (and then a column) of the number of companies close to each of them in 2000 meters

lst =[] 
rad = 2000 # meters

for e in range(len(df)):
    num = findNear(lista[e],rad)
    lst.append(len(num))
print(lst)

df["nearest_offices"] = lst

df_2000 = df.sort_values(by = "nearest_offices", ascending = False)
df_2000.reset_index(inplace=True)

display(df.head())

[28, 32, 1, 14, 27, 7, 1, 5, 4, 1, 2, 7, 4, 2, 12, 2, 1, 6, 1, 6, 28, 27, 1, 3, 6, 27, 1, 1, 12, 1, 1, 14, 3, 27, 27, 1, 8, 2, 1, 1, 1, 1, 2, 7, 1, 1, 27, 5, 1, 3, 1, 5, 5, 2, 4, 2, 1, 28, 13, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 11, 3, 2, 7, 4, 4, 1, 1, 13, 3, 1, 3, 6, 13, 27, 1, 1, 1, 1, 28, 1, 1, 11, 1, 13, 1, 1, 3, 1, 8, 1, 1, 2, 2, 14, 1, 2, 12, 7, 9, 1, 11, 1, 2, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 2, 27, 12, 2, 1, 1, 1, 1, 3, 11, 4, 1, 1, 1, 1, 1, 2, 13, 7, 36, 25, 2, 27, 2, 17, 5, 27, 14, 2, 8, 25, 28, 5, 1, 27, 8, 25, 2, 1, 1, 1, 4, 10, 2, 2, 3, 28, 2, 2, 4, 1, 1, 1, 1, 27, 2, 1, 11, 1, 2, 1, 1, 1, 3, 2, 3, 2, 27, 2, 1, 1, 1, 27, 5, 3, 2, 1, 3, 1, 4, 1, 3, 1, 1, 2, 1, 3, 1, 1, 1, 3, 1, 1, 27, 3, 1, 2, 27, 1, 1, 1, 1, 6, 2, 1, 1, 1, 1, 13, 1, 1, 5, 1, 1, 14]


,index,_id,category_code,country_code,founded_year,geoloc,latitude,longitude,name,number_of_employees,total_money_raised,total_offices,nearest_offices
0,51,5d277baf5370f8e6fc94d8cd,enterprise,USA,2006,"{'type': 'Point', 'coordinates': [-122.158804,...",37.439380,-122.158804,Gigya,230,$69.8M,1,28
1,20,5d277baf5370f8e6fc94d8ae,web,USA,2006,"{'type': 'Point', 'coordinates': [-122.39071, ...",37.779507,-122.390710,Eventbrite,200,$140M,1,32
2,167,5d277baf5370f8e6fc94d941,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-121.967223,...",37.369445,-121.967223,Apigee,65,$107M,1,1
3,26,5d277baf5370f8e6fc94d8b4,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-104.988123,...",39.755092,-104.988123,NewsGator,90,$39.3M,1,14
4,190,5d277baf5370f8e6fc94d958,advertising,USA,2006,"{'type': 'Point', 'coordinates': [-105.278785,...",40.058448,-105.278785,Intela,70,$13.3M,4,27


In [35]:
df_2000.to_csv("for_api_2000.csv")

In [36]:
# Let's see on the map where are the top 10 companies for 1500 m radius

In [44]:
df_1500_10 = df.head(10)
#df_500_10.reset_index(inplace = True)

def points(col_lat, col_lon): #insert values as (df[col_lat], df[col_lon])
    statmad = df_1500_10[['latitude', 'longitude']]
    statmadlist = statmad.values.tolist()
    popup = df_1500_10[['name']]
    statmap = folium.Map() 
    for point in range(0, len(statmadlist)):
        folium.Marker(statmadlist[point], popup=popup.iloc[point,0]).add_to(statmap)
    return statmap    

points(df_1500_10['latitude'], df_1500_10['longitude'])